In [0]:
#loading dataset file from drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!unzip "/content/gdrive/My Drive/data_csv.zip"
# !unzip "/content/gdrive/My Drive/challenge_set.zip"


In [0]:
#installing spark and java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.3.3/spark-2.3.3-bin-hadoop2.7.tgz
!tar xf spark-2.3.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
#locating spark and java
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.3-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import StorageLevel
spark = SparkSession.builder.appName('music recommendation system').config("spark.driver.memory","15g").config("spark.executor.memory","15g").getOrCreate()
df=spark.read.format("csv").option("header", "true").load("*.csv")


In [0]:
df.show()

In [0]:
# from pyspark import SparkConf,SparkContext
# conf = SparkConf()
# sc = SparkContext(conf=conf)
# pickleRdd = sparkpickle('dataframe').collect()
# df = spark.createDataFrame(pickleRdd)
from pyspark.sql.types import *
from pyspark.sql.functions import lit
from pyspark.sql.functions import col
# Adding the rating column 
all_data = df.select("trackid","artist_name","track_name", "pid").withColumn("rating", lit(1))



In [0]:
from pyspark.ml.feature import StringIndexer,StringIndexerModel
# stringIndexer = StringIndexer(inputCol="trackid", outputCol="songid", handleInvalid="error",stringOrderType='frequencyDesc')
# model = stringIndexer.fit(all_data)
model=StringIndexerModel.load('/content/gdrive/My Drive/stringindexmodel.h5')
df = model.transform(all_data)
#df.show()

In [0]:
all_data.unpersist()

DataFrame[trackid: string, artist_name: string, track_name: string, pid: string, rating: int]

In [0]:
from pyspark.sql.types import IntegerType
df= df.withColumn("songid1", df["songid"].cast(IntegerType()))
df=df.withColumn("pid1",df["pid"].cast(IntegerType()))
train, test = df.randomSplit([0.9, 0.1], seed = 2019)
# print("Training Dataset Count: " + str(train.count()))
# print("Test Dataset Count: " + str(test.count()))
#rating_df.show()

In [0]:
rating_df=train.select("pid1","songid1","rating").filter(train.pid1.isNotNull())
test=test.select("pid1","songid1","rating").filter(test.pid1.isNotNull())
# df=df.select("pid1","songid1","rating").filter(df.pid1.isNotNull())

In [0]:
##TRAINING AND SAVING MODEL
from pyspark.ml.recommendation import ALS
als=ALS(rank=5,maxIter=50,regParam=0.2,implicitPrefs=True, userCol="pid1", itemCol="songid1", ratingCol="rating",
          coldStartStrategy="drop")
model=als.fit(rating_df)
model.save('/content/gdrive/My Drive/ALSmodel2.h5')

In [0]:
#LOADING MODEL
from pyspark.ml.recommendation import ALSModel
model = ALSModel.load('/content/gdrive/My Drive/ALSmodel1.h5')

In [0]:
#RECOMMEND FOR TEST DATASET
# user_subset = rating_df.where(rating_df.pid1<100000 )
userRecs = model.recommendForUserSubset(test, 100)

In [0]:
from pyspark.sql.functions import explode
top_k=userRecs.withColumn("Recommendations", explode(userRecs.recommendations))

In [0]:
top_k=top_k.select('pid1', 'Recommendations.*')
top_k=top_k.select('pid1','songid1')
# top_k.show()

In [0]:
from pyspark.sql.types import DoubleType
top_k= top_k.withColumn("songid1", top_k["songid1"].cast(DoubleType()))


In [0]:
# df.createOrReplaceTempView("dataframe")
train.createOrReplaceTempView("dataframe")
top_k.createOrReplaceTempView("results")
df2 = spark.sql("SELECT dataframe.songid,dataframe.track_name,dataframe.artist_name from dataframe where dataframe.songid IN(SELECT results.songid1 from results)").persist(StorageLevel.DISK_ONLY)
spark.catalog.dropTempView("dataframe")
spark.catalog.dropTempView("results")


In [0]:
final=top_k.join(df2,top_k.songid1==df2.songid).dropDuplicates()


In [0]:
from pyspark.sql.functions import *
final=final.select("pid1","track_name","artist_name").withColumnRenamed("track_name", "Recommendation").orderBy('pid1')
# final.show()

In [0]:

# df3=df.select("pid1","track_name","artist_name")
df3=df.select("pid1","track_name","artist_name").filter(df.pid1.isNotNull()).orderBy("pid1")

#selecting pid 0 to 99999 for evaluation
# df3=df3.where(df3.pid1<100000)
#df3.show()

In [0]:
from pyspark.sql import functions as f
final=final.groupBy(f.col('pid1')).agg(f.collect_set('Recommendation').alias('Recommendation'),f.collect_set('artist_name').alias('Rartist_name'))
# itcount=topk.select('*',f.size('Recommendation').alias('track_intersection'),f.size('artist_name').alias('artist_intersection'))

In [0]:
df3=df3.groupBy(f.col('pid1')).agg(f.collect_set('track_name').alias('track_name'),f.collect_set('artist_name').alias('artist_name'))
# gt=counts.select('*',f.size('track_name').alias('track_count'),f.size('artist_name').alias('artist_count'))

In [0]:
df3.createOrReplaceTempView("t1")
final.createOrReplaceTempView("t2")
temp = spark.sql("SELECT t1.pid1,t1.track_name,t1.artist_name,t2.Recommendation,t2.Rartist_name from t1,t2 where t1.pid1==t2.pid1")
spark.catalog.dropTempView("t1")
spark.catalog.dropTempView("t2")
# temp=final.join(df3,"pid1").persist(StorageLevel.DISK_ONLY)

# .withColumn('Rprecision_track',(f.col('track_intersection') / f.col('track_count')))
# res=temp.withColumn('Rprecision_artist',(f.col('artist_intersection')/f.col('artist_count')))
# temp.take(3)

In [0]:
from pyspark.mllib.evaluation import RankingMetrics
#NDCG SCORE
rddtrack=temp.rdd.map(lambda x:(x[1],x[3]))
metrics=RankingMetrics(rddtrack)
metrics.ndcgAt(100)

0.06814588324799176

In [0]:
#Mean Average Precision
metrics.meanAveragePrecision

In [0]:
#R-Precision- Logic one
!pip install pyarrow
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
topk1=topk.toPandas()
original=df3.toPandas()

import numpy as np
def R_prec():
  total=0
  for i in range(100000):
    d=len(original[original['pid1']==i])
    it=np.intersect1d(topk1[topk1['pid1']==i]['Recommendation'],original[original['pid1']==i]['track_name'])
    it1=np.intersect1d(topk1[topk1['pid1']==i]['artist_name'],original[original['pid1']==i]['artist_name'])
    n=len(it)/d
    n1=len(it1)/d
    total+=n+0.25*n1
  return (total/100000)
rp=R_prec()
rp

In [0]:

#R-Precision- Logic 2
counts=df3.groupBy(col('pid1')).count()
import numpy as np
def R_prec():
  n=0
  n1=0
  for i in range(3000):
    num=counts.select("count").where(counts.pid1==i).collect()[0]['count']
    Gt=[row.track_name for row in df3.select('track_name').where(df3.pid1==i).collect()]
    Ga=[row.artist_name for row in df3.select('artist_name').where(df3.pid1==i).collect()]
    Rt=[row.Recommendation for row in topk.select('Recommendation').where(topk.pid1==i).collect()]
    Ra=[row.artist_name for row in topk.select('artist_name').where(topk.pid1==i).collect()]
    it=np.intersect1d(Rt,Gt)
    it1=np.intersect1d(Ra,Ga)
    n=len(it)/num
    n1=len(it1)/num
    yield (n+0.25*n1)
rp=[j for j in R_prec()]
np.mean(rp) 

In [0]:
#R-precision-Logic 3
counts=temp.groupBy(col('pid1')).count()
import numpy as np
pid=[row.pid1 for row in temp.select('pid1').collect()]
def Rprec():
  for i in pid:
    num=counts.select("count").where(counts.pid1==i).collect()[0]['count']
    it=np.intersect1d(temp.select('Recommendation').where(temp.pid1==i).collect(),temp.select('track_name').where(temp.pid1==i).collect())
    it1=np.intersect1d(temp.select('Rartist_name').where(temp.pid1==i).collect(),temp.select('artist_name').where(temp.pid1==i).collect())
    n=len(it)/num
    n1=len(it1)/num
    yield (n+0.25*n1)
rp=[j for j in Rprec()]
np.mean(rp)

In [0]:
# import numpy as np
# import pandas as pd
# Gt=[(row.pid1,row.track_name,row.artist_name) for row in df3.collect()]
# Rt=[(row.pid1,row.Recommendation,row.artist_name) for row in topk.collect()]
# # Gtruth= pd.DataFrame(Gt,columns=['pid','track_name','artist_name'])
# # reco=pd.DataFrame(Rt,columns=['pid','Recommendation','artist_name'])

# def R_prec():
#   n=0
#   n1=0
#   m=[]
#   for i in range(200000):
#     it=np.intersect1d(reco[reco['pid']==i]['Recommendation'],Gtruth[Gtruth['pid']==i]['track_name'])
#     it1=np.intersect1d(reco[reco['pid']==i]['artist_name'],Gtruth[Gtruth['pid']==i]['artist_name'])
#     n=len(it)/len(Gtruth[Gtruth['pid']==i])
#     n1=len(it1)/len(Gtruth[Gtruth['pid']==i])
#     m.append(n+0.25*n1)
#   return m
# rp=R_prec()
# np.mean(rp)

In [0]:
# from pyspark.sql import functions as F
# df4=df3.groupby("pid1").agg(F.collect_set("track_name").alias("track_name"),F.collect_set("artist_name").alias("artist_name"))
# topk1=topk.groupby("pid1").agg(F.collect_set("Recommendation").alias("Recommendation"),F.collect_set("artist_name").alias("artist_name"))
# counts=df3.groupBy(col('pid1')).count()
# def R_prec():
#   n=0
#   n1=0
#   for i in range(1000):
#     num=counts.where(counts.pid1==i).collect()[0]['count']
#     it=np.intersect1d(topk1.where(topk1.pid1==0).collect()[0]['Recommendation'],df4.where(df4.pid1==0).collect()[0]['track_name'])
#     it1=np.intersect1d(topk1.where(topk1.pid1==0).collect()[0]['artist_name'],df4.where(df4.pid1==0).collect()[0]['artist_name'])
#     n=len(it)/num
#     n1=len(it1)/num
#     yield (n+0.25*n1)
# rp=[]
# rprec=R_prec()
# for i in rprec:
#   rp.append(i)
# np.mean(rp) 




In [0]:
#R-precision - logic 4(optimized)
!pip install handyspark
from handyspark import *
from pyspark.sql import functions as F
import numpy as np
recommended=temp.toHandy()

def Rprec():
  for i,j,k,l in zip(recommended.cols['Recommendation'][:],recommended.cols['track_name'][:],recommended.cols['Rartist_name'][:],recommended.cols['artist_name'][:]):
    it=np.intersect1d(i,j)
    it1=np.intersect1d(k,l)
    yield ((len(it)+0.25*len(it1))/len(j))
rp=[r for r in Rprec()]
np.mean(rp)